## Topic Modelling using LDA
---

In [ ]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Kaggle/data_btm.csv')
data.head()

,Unnamed: 0,url,name,online_order,book_table,rate,location,rest_type,cuisines,cost(for 2),dining_type,city,Sentiment_Score,Sentiment_Class,cleaned_reviews,word_embeddings_of_reviews
0,54161,https://www.zomato.com/bangalore/navya-foods-h...,4124,1,0,0.286423,22,1.278466,1.001651,-0.582052,-0.486294,3,0.9002,4,boneless biriyani good like much homely foods ...,[[ 7.27092862e-01 2.27685332e+00 3.07956123e...
1,69129,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.7885,3,x ordered place multiple time love food favour...,[[ 3.63598323e+00 -4.17936325e-01 2.64801741e...
2,69130,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.9623,4,new outlet hsr serve best food hsr guess try a...,[[ 2.5347857 0.22424214 1.9644408 ... 3.8...
3,69131,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.8297,4,place order aloo paratha dal makhani box parat...,[[ 3.2815177e+00 2.9132953e+00 -1.1734662e+00...
4,69132,https://www.zomato.com/bangalore/the-paratha-c...,6375,1,0,0.845013,22,-0.203611,0.647824,-0.354182,-0.486294,4,0.9854,4,go zomato find place place order aloo paratha ...,[[ 2.0472045 0.7242372 3.7346416 ... 1.9...


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(max_features=700, stop_words='english')
X = vectorizer.fit_transform(data['cleaned_reviews'])

lda_model = LatentDirichletAllocation(n_components=500, random_state=42)  # Adjust the number of topics as needed
lda_model.fit(X)

# Extract the main topics
def get_main_topics(model, vectorizer, n_words=100):
    keywords = []
    feature_names = vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
        top_keywords_idx = topic.argsort()[:-n_words - 1:-1]
        keywords.append([feature_names[i] for i in top_keywords_idx])
    return keywords

main_topics = get_main_topics(lda_model, vectorizer)

In [ ]:
main_topics[0]

['order',
 'food',
 'taste',
 'chicken',
 'bad',
 'rice',
 'good',
 'delivery',
 'quantity',
 'quality']

In [ ]:
import pickle
file_path = '/content/drive/MyDrive/Kaggle/lda_model.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(lda_model, file)

In [ ]:
main_topics_series = pd.Series(main_topics, name='main_topics')
data_with_main_topics = pd.concat([data, main_topics_series], axis=1)
selected_columns = ['url', 'Sentiment_Score', 'Sentiment_Class', 'cleaned_reviews', 'main_topics']

data_with_topics = data_with_main_topics[selected_columns]

In [ ]:
data_with_topics.shape

(64408, 5)

In [ ]:
import pickle
file_path = '/content/drive/MyDrive/Kaggle/data_with_topics_new.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(data_with_topics, file)

In [ ]:
data = pd.read_pickle('/content/drive/MyDrive/Kaggle/data_with_topics_new.pkl')
data.head()

,url,Sentiment_Score,Sentiment_Class,cleaned_reviews,main_topics
0,https://www.zomato.com/bangalore/navya-foods-h...,0.9002,4,boneless biriyani good like much homely foods ...,"[cream, ice, try, place, good, like, flavor, t..."
1,https://www.zomato.com/bangalore/the-paratha-c...,0.7885,3,x ordered place multiple time love food favour...,"[noodle, order, home, biryani, place, noon, de..."
2,https://www.zomato.com/bangalore/the-paratha-c...,0.9623,4,new outlet hsr serve best food hsr guess try a...,"[biriyani, chicken, taste, great, place, try, ..."
3,https://www.zomato.com/bangalore/the-paratha-c...,0.8297,4,place order aloo paratha dal makhani box parat...,"[bowl, rice, order, chicken, serve, like, plac..."
4,https://www.zomato.com/bangalore/the-paratha-c...,0.9854,4,go zomato find place place order aloo paratha ...,"[raita, good, biryani, quality, food, order, d..."


In [ ]:
data.shape

(64408, 6)

In [ ]:
data['main_topics'][0]

['order',
 'food',
 'chicken',
 'taste',
 'biryani',
 'good',
 'bad',
 'delivery',
 'rice',
 'restaurant']

### Add sentiment of main topics found from LDA using Vader and storing it in dictionary

In [ ]:
import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize Vader Sentiment Analyzer
sid = SentimentIntensityAnalyzer()

# Assuming data is your DataFrame with the 'main_topics' and 'cleaned_reviews' columns
main_topics_list = data['main_topics']
cleaned_reviews_list = data['cleaned_reviews']

# Determine sentiment of words in main_topics based on cleaned_reviews and store in a dictionary format
word_sentiments_dicts = []

# Iterate over the elements in the 'main_topics' and 'cleaned_reviews' columns
for topic_list, review in zip(main_topics_list, cleaned_reviews_list):
    # Check if the element in main_topics is a list
    if isinstance(topic_list, list):
        word_sentiments_dict = {}
        for word in topic_list:
            # Get sentiment score for the word based on the review
            sentiment_score = sid.polarity_scores(review)['compound']

            # Add word-sentiment pair to the dictionary
            word_sentiments_dict[word] = sentiment_score

        # Append the dictionary to the list
        word_sentiments_dicts.append(word_sentiments_dict)
    else:
        # If the element in main_topics is not a list, append an empty dictionary
        word_sentiments_dicts.append({})

# Add word sentiments to the DataFrame as a new column
data['word_sentiments_dict'] = word_sentiments_dicts

data.head()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


,url,Sentiment_Score,Sentiment_Class,cleaned_reviews,main_topics,word_sentiments_dict
0,https://www.zomato.com/bangalore/navya-foods-h...,0.9002,4,boneless biriyani good like much homely foods ...,"[cream, ice, try, place, good, like, flavor, t...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625..."
1,https://www.zomato.com/bangalore/the-paratha-c...,0.7885,3,x ordered place multiple time love food favour...,"[noodle, order, home, biryani, place, noon, de...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0...."
2,https://www.zomato.com/bangalore/the-paratha-c...,0.9623,4,new outlet hsr serve best food hsr guess try a...,"[biriyani, chicken, taste, great, place, try, ...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste..."
3,https://www.zomato.com/bangalore/the-paratha-c...,0.8297,4,place order aloo paratha dal makhani box parat...,"[bowl, rice, order, chicken, serve, like, plac...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87..."
4,https://www.zomato.com/bangalore/the-paratha-c...,0.9854,4,go zomato find place place order aloo paratha ...,"[raita, good, biryani, quality, food, order, d...","{'raita': 0.98, 'good': 0.98, 'biryani': 0.98,..."


In [ ]:
data['url'].nunique()

2194

In [ ]:
data['word_sentiments_dict'][0]

In [ ]:
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

# Initialize Vader Sentiment Analyzer
sid = SentimentIntensityAnalyzer()

# Assuming data is your DataFrame with the 'main_topics' and 'cleaned_reviews' columns
main_topics_list = data['main_topics']
cleaned_reviews_list = data['cleaned_reviews']

# Determine sentiment of words in main_topics based on cleaned_reviews and store in a list of dictionaries
word_sentiments_dicts = []

# Iterate over the elements in the 'main_topics' and 'cleaned_reviews' columns
for topic_list, review in zip(main_topics_list, cleaned_reviews_list):
    word_sentiments_dict = {}
    # Check if the element in main_topics is a list
    if isinstance(topic_list, list):
        for word in topic_list:
            # Get sentiment score for the word based on its occurrence in the review
            sentiment_score = sid.polarity_scores(word)['compound']

            # Add word-sentiment pair to the dictionary
            word_sentiments_dict[word] = sentiment_score

        # Append the dictionary to the list
        word_sentiments_dicts.append(word_sentiments_dict)
    else:
        # If the element in main_topics is not a list, append an empty dictionary
        word_sentiments_dicts.append({})

# Add word sentiments to the DataFrame as a new column
data['word_sentiments_dict_new'] = word_sentiments_dicts

data.head()

,url,Sentiment_Score,Sentiment_Class,cleaned_reviews,main_topics,word_sentiments_dict,word_sentiments_dict_new
0,https://www.zomato.com/bangalore/navya-foods-h...,0.9002,4,boneless biriyani good like much homely foods ...,"[cream, ice, try, place, good, like, flavor, t...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625...","{'cream': 0.0, 'ice': 0.0, 'try': 0.0, 'place'..."
1,https://www.zomato.com/bangalore/the-paratha-c...,0.7885,3,x ordered place multiple time love food favour...,"[noodle, order, home, biryani, place, noon, de...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0....","{'noodle': 0.0, 'order': 0.0, 'home': 0.0, 'bi..."
2,https://www.zomato.com/bangalore/the-paratha-c...,0.9623,4,new outlet hsr serve best food hsr guess try a...,"[biriyani, chicken, taste, great, place, try, ...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste...","{'biriyani': 0.0, 'chicken': 0.0, 'taste': 0.0..."
3,https://www.zomato.com/bangalore/the-paratha-c...,0.8297,4,place order aloo paratha dal makhani box parat...,"[bowl, rice, order, chicken, serve, like, plac...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87...","{'bowl': 0.0, 'rice': 0.0, 'order': 0.0, 'chic..."
4,https://www.zomato.com/bangalore/the-paratha-c...,0.9854,4,go zomato find place place order aloo paratha ...,"[raita, good, biryani, quality, food, order, d...","{'raita': 0.98, 'good': 0.98, 'biryani': 0.98,...","{'raita': 0.0, 'good': 0.4404, 'biryani': 0.0,..."


In [ ]:
data['word_sentiments_dict_new'][0]

In [ ]:
import copy

# Initialize an empty list to store the final sentiment dictionaries
final_sentiments = []

# Iterate through each row in the dataset
for index, row in data.iterrows():
    dict_new = row['word_sentiments_dict_new']
    dict_old = row['word_sentiments_dict']

    # Merge the dictionaries while giving priority to values from dict_new
    if dict_old != {}:
      final_dict = copy.deepcopy(dict_old)
    for key, value in dict_new.items():
        final_dict[key] = value if value != 0 else final_dict.get(key, 0)

    # Append the merged dictionary to the list
    final_sentiments.append(final_dict)

# Add the list of merged dictionaries as a new column
data['final_sentiment_score_of_topics'] = final_sentiments

# Print the first few rows to verify
data.head()

,url,Sentiment_Score,Sentiment_Class,cleaned_reviews,main_topics,word_sentiments_dict,word_sentiments_dict_new,final_sentiment_score_of_topics,result,weighted_average
0,https://www.zomato.com/bangalore/navya-foods-h...,0.9002,4,boneless biriyani good like much homely foods ...,"[cream, ice, try, place, good, like, flavor, t...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625...","{'cream': 0.0, 'ice': 0.0, 'try': 0.0, 'place'...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625...","{'cream': 0.7764225, 'ice': 0.7764225, 'try': ...","{'cream': 0.6989355345, 'ice': 0.6989355345, '..."
1,https://www.zomato.com/bangalore/the-paratha-c...,0.7885,3,x ordered place multiple time love food favour...,"[noodle, order, home, biryani, place, noon, de...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0....","{'noodle': 0.0, 'order': 0.0, 'home': 0.0, 'bi...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0....","{'noodle': 0.6035178999999999, 'order': 0.6035...","{'noodle': 0.47587386414999994, 'order': 0.475..."
2,https://www.zomato.com/bangalore/the-paratha-c...,0.9623,4,new outlet hsr serve best food hsr guess try a...,"[biriyani, chicken, taste, great, place, try, ...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste...","{'biriyani': 0.0, 'chicken': 0.0, 'taste': 0.0...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste...","{'biriyani': 0.92303816, 'chicken': 0.92303816...","{'biriyani': 0.888239621368, 'chicken': 0.8882..."
3,https://www.zomato.com/bangalore/the-paratha-c...,0.8297,4,place order aloo paratha dal makhani box parat...,"[bowl, rice, order, chicken, serve, like, plac...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87...","{'bowl': 0.0, 'rice': 0.0, 'order': 0.0, 'chic...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87...","{'bowl': 0.72532374, 'rice': 0.72532374, 'orde...","{'bowl': 0.6018011070779999, 'rice': 0.6018011..."
4,https://www.zomato.com/bangalore/the-paratha-c...,0.9854,4,go zomato find place place order aloo paratha ...,"[raita, good, biryani, quality, food, order, d...","{'raita': 0.98, 'good': 0.98, 'biryani': 0.98,...","{'raita': 0.0, 'good': 0.4404, 'biryani': 0.0,...","{'raita': 0.98, 'good': 0.4404, 'biryani': 0.9...","{'raita': 0.965692, 'good': 0.4339701600000000...","{'raita': 0.9515928968, 'good': 0.427634195664..."


In [ ]:
data.shape

(64408, 9)

In [ ]:
import pandas as pd

def multiply_sentiment_score(row):
    sentiment_score = row['Sentiment_Score']
    final_sentiment_score_of_topics = row['final_sentiment_score_of_topics']
    if final_sentiment_score_of_topics:
        result = {key: value * sentiment_score for key, value in final_sentiment_score_of_topics.items()}
    else:
        result = {}
    return result

# Apply the function to each row to create the new column
data['result'] = data.apply(multiply_sentiment_score, axis=1)
data.head()

,url,Sentiment_Score,Sentiment_Class,cleaned_reviews,main_topics,word_sentiments_dict,word_sentiments_dict_new,final_sentiment_score_of_topics,result,weighted_average
0,https://www.zomato.com/bangalore/navya-foods-h...,0.9002,4,boneless biriyani good like much homely foods ...,"[cream, ice, try, place, good, like, flavor, t...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625...","{'cream': 0.0, 'ice': 0.0, 'try': 0.0, 'place'...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625...","{'cream': 0.7764225, 'ice': 0.7764225, 'try': ...","{'cream': 0.6989355345, 'ice': 0.6989355345, '..."
1,https://www.zomato.com/bangalore/the-paratha-c...,0.7885,3,x ordered place multiple time love food favour...,"[noodle, order, home, biryani, place, noon, de...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0....","{'noodle': 0.0, 'order': 0.0, 'home': 0.0, 'bi...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0....","{'noodle': 0.6035178999999999, 'order': 0.6035...","{'noodle': 0.47587386414999994, 'order': 0.475..."
2,https://www.zomato.com/bangalore/the-paratha-c...,0.9623,4,new outlet hsr serve best food hsr guess try a...,"[biriyani, chicken, taste, great, place, try, ...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste...","{'biriyani': 0.0, 'chicken': 0.0, 'taste': 0.0...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste...","{'biriyani': 0.92303816, 'chicken': 0.92303816...","{'biriyani': 0.888239621368, 'chicken': 0.8882..."
3,https://www.zomato.com/bangalore/the-paratha-c...,0.8297,4,place order aloo paratha dal makhani box parat...,"[bowl, rice, order, chicken, serve, like, plac...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87...","{'bowl': 0.0, 'rice': 0.0, 'order': 0.0, 'chic...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87...","{'bowl': 0.72532374, 'rice': 0.72532374, 'orde...","{'bowl': 0.6018011070779999, 'rice': 0.6018011..."
4,https://www.zomato.com/bangalore/the-paratha-c...,0.9854,4,go zomato find place place order aloo paratha ...,"[raita, good, biryani, quality, food, order, d...","{'raita': 0.98, 'good': 0.98, 'biryani': 0.98,...","{'raita': 0.0, 'good': 0.4404, 'biryani': 0.0,...","{'raita': 0.98, 'good': 0.4404, 'biryani': 0.9...","{'raita': 0.965692, 'good': 0.4339701600000000...","{'raita': 0.9515928968, 'good': 0.427634195664..."


In [ ]:
import pandas as pd
from collections import defaultdict
# Function to calculate the weighted average for each key
def calculate_weighted_average(row):
    result = row['result']
    sentiment_score = row['Sentiment_Score']
    if not result:
        return {}

    weighted_sum = defaultdict(float)
    total_frequency = defaultdict(int)

    # Calculate weighted sum and total frequency for each key
    for key, value in result.items():
        weighted_sum[key] += value * sentiment_score
        total_frequency[key] += 1

    # Calculate weighted average for each key
    weighted_average = {key: weighted_sum[key] / total_frequency[key] for key in weighted_sum}

    return weighted_average

# Apply the function to each row to create the new column
data['weighted_average'] = data.apply(calculate_weighted_average, axis=1)

data.head()

,url,Sentiment_Score,Sentiment_Class,cleaned_reviews,main_topics,word_sentiments_dict,word_sentiments_dict_new,final_sentiment_score_of_topics,result,weighted_average
0,https://www.zomato.com/bangalore/navya-foods-h...,0.9002,4,boneless biriyani good like much homely foods ...,"[cream, ice, try, place, good, like, flavor, t...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625...","{'cream': 0.0, 'ice': 0.0, 'try': 0.0, 'place'...","{'cream': 0.8625, 'ice': 0.8625, 'try': 0.8625...","{'cream': 0.7764225, 'ice': 0.7764225, 'try': ...","{'cream': 0.6989355345, 'ice': 0.6989355345, '..."
1,https://www.zomato.com/bangalore/the-paratha-c...,0.7885,3,x ordered place multiple time love food favour...,"[noodle, order, home, biryani, place, noon, de...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0....","{'noodle': 0.0, 'order': 0.0, 'home': 0.0, 'bi...","{'noodle': 0.7654, 'order': 0.7654, 'home': 0....","{'noodle': 0.6035178999999999, 'order': 0.6035...","{'noodle': 0.47587386414999994, 'order': 0.475..."
2,https://www.zomato.com/bangalore/the-paratha-c...,0.9623,4,new outlet hsr serve best food hsr guess try a...,"[biriyani, chicken, taste, great, place, try, ...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste...","{'biriyani': 0.0, 'chicken': 0.0, 'taste': 0.0...","{'biriyani': 0.9592, 'chicken': 0.9592, 'taste...","{'biriyani': 0.92303816, 'chicken': 0.92303816...","{'biriyani': 0.888239621368, 'chicken': 0.8882..."
3,https://www.zomato.com/bangalore/the-paratha-c...,0.8297,4,place order aloo paratha dal makhani box parat...,"[bowl, rice, order, chicken, serve, like, plac...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87...","{'bowl': 0.0, 'rice': 0.0, 'order': 0.0, 'chic...","{'bowl': 0.8742, 'rice': 0.8742, 'order': 0.87...","{'bowl': 0.72532374, 'rice': 0.72532374, 'orde...","{'bowl': 0.6018011070779999, 'rice': 0.6018011..."
4,https://www.zomato.com/bangalore/the-paratha-c...,0.9854,4,go zomato find place place order aloo paratha ...,"[raita, good, biryani, quality, food, order, d...","{'raita': 0.98, 'good': 0.98, 'biryani': 0.98,...","{'raita': 0.0, 'good': 0.4404, 'biryani': 0.0,...","{'raita': 0.98, 'good': 0.4404, 'biryani': 0.9...","{'raita': 0.965692, 'good': 0.4339701600000000...","{'raita': 0.9515928968, 'good': 0.427634195664..."


In [ ]:
data['weighted_average'][0]

### Combining the dictionaries for each URL to generate tags
---

In [ ]:
import pandas as pd


# Convert data to a DataFrame
df = pd.DataFrame(data['url'], columns=['url'])

# Assign 'final_sentiment_score_of_topics' list as another column
df['final_sentiment_score_of_topics'] = data['final_sentiment_score_of_topics']

# Initialize an empty dictionary to store combined values
combined_dict = {}

# Iterate through each row in the DataFrame
for index, row in df.iterrows():
    url = row['url']
    final_sentiment_score_of_topics = row['final_sentiment_score_of_topics']

    # Update the combined_dict with the values from final_sentiment_score_of_topics
    if url in combined_dict:
        for key, value in final_sentiment_score_of_topics.items():
            combined_dict[url][key] = combined_dict[url].get(key, 0) + value
    else:
        combined_dict[url] = word_sentiments_dict.copy()

# Convert the combined_dict to a DataFrame
combined_df = pd.DataFrame(combined_dict.items(), columns=['url', 'combined_dict'])
combined_df.head()

,url,combined_dict
0,https://www.zomato.com/bangalore/navya-foods-h...,{}
1,https://www.zomato.com/bangalore/the-paratha-c...,"{'biriyani': 0.9592, 'chicken': 4.6886, 'taste..."
2,https://www.zomato.com/bangalore/new-punjabi-d...,"{'ok': 0.296, 'food': 2.8449, 'ambience': 1.37..."
3,https://www.zomato.com/bangalore/xo-belgian-wa...,"{'soft': 5.000500000000001, 'chicken': 11.7479..."
4,https://www.zomato.com/bangalore/bathinda-junc...,"{'ingredient': 2.1437, 'quite': 10.97580000000..."


In [ ]:
combined_df['combined_dict'].apply(lambda x: len(x) == 0).sum()

449

In [ ]:
file_path = '/content/drive/MyDrive/Kaggle/combined.pkl'
with open(file_path, 'wb') as file:
    pickle.dump(combined_df, file)

In [ ]:
average_sentiment = data.groupby('url')['Sentiment_Score'].mean().reset_index()
combined_df = pd.merge(combined_df, average_sentiment, on='url')

In [ ]:
combined_df.head()

,url,combined_dict,Sentiment_Score
0,https://www.zomato.com/bangalore/navya-foods-h...,{},0.900200
1,https://www.zomato.com/bangalore/the-paratha-c...,"{'biriyani': 0.9592, 'chicken': 4.6886, 'taste...",0.834815
2,https://www.zomato.com/bangalore/new-punjabi-d...,"{'ok': 0.296, 'food': 2.8449, 'ambience': 1.37...",0.500471
3,https://www.zomato.com/bangalore/xo-belgian-wa...,"{'soft': 5.000500000000001, 'chicken': 11.7479...",0.876121
4,https://www.zomato.com/bangalore/bathinda-junc...,"{'ingredient': 2.1437, 'quite': 10.97580000000...",0.789211


In [ ]:
from google.colab import files
combined_df.to_csv('combined.csv')
files.download('combined.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>